In [ ]:
#Getting started with BQL
Import bql library: import bql 
Write a query: request = """get(PX_HIGH - PX_LOW) for ([''AAPL US Equity])"""
Execute a query: response = bq.execute(request)   
                 
    Define variables to call in query: 
    security = 'AAPL US Equity'
    high = bq.data.px_high()
    low = bq.data.px_low()
    request = bql.Request(security, high - low)
    response = bq.execute(request)

Grouping data: #By default, when you create abQL request, the data returned is grouped at security level. 
                     
    import bql 
    bq = bql.Service() #Instantiate an object to interface with BQL services
    univ = ['sec1', 'sec2', 'sec3']
    fcf_yield = bq.data.free_cash_flow_yield(fill = 'prev') #what is fill 'prev'
    request = blq.Request(univ, fcf_yield)  #Generate the request using the universe variable and data item  
    response = bq. execute(request) 
    response[0].df() #Displays response as df
    
Get avg fcf for a security: avg_fcf_yield = bq.data.free_cash_flow_yield(fill='prev').avg()            
Get avg fcf across securities: avg_fcf_yield = bq.data.free_cash_flow_yield(fill='prev').group().avg()
Get avvg fcf across GICS sector in index: avg_fcf_yield = 
 bq.data.free_cash_flow_yield(fill='prev').group(grouping_item).avg()

In [ ]:
# BQL 
# Powerful API that lets you retrieve Bloomberg data and transform it directly in the Bloomberg Cloud.  
# What is it for: calculations, statistical analysis, screening applied to Bloomberg's normalized, point-in-time data 

ATTENTION: BQL POUR SERVICE, REQUEST, COMBINED_DF / BQ SINON
Flow: import > service > request > execute > .combined_df()

Importable libraries
#Access full list in Help > BQuant Packages 
- Data Analysis: nltk, numexpr, pandas, scipy, scikit-learn, numpy, algopy, bqfunc # Bbg library for financial closed-form expressions 
- Viz: bqplot, bqplot-extra, seaborn, pillow, matplotlib, ipywidgets, ipyleaflet, cycler, bqviz
- Watch out: ipyleaflet # Jupyter / Leaflet bridge enabling interactive maps in the notebook

Concepts 
- data item (class): tell the query to pull a specific type of data from Bloom. Bloomberg allows for thousands of data items. 
- ID (class) : 'TSLA US Equity' - always separated by , and contained within []
- dates (param): either absolute YYYY-MM-DD or relative (-1D, -2D). If not specified, default value = 0D 
- Service (class): used to instantiate an object with interface to BQL service. The Service class contains methods to access BQL data items, functions, universe handlers
- Request (class): used to generate BQL requests
- Response (class): defines the format of responses to BQL requests. An instance of this class is returned by Service.execute() as the result of the request exec. 
#The response object contains only one SingleItemResponse instance for each item requested in the GET part of the query. Several data items asked = possible to loop through responses.
# NOTE: ONLY APPLY .df() WITH SINGLE ITEM RESPONSES 

    Loop through responses
    bq = bql.Service()
    response = bq.execute(['IBM US Equity'], [bq.data.px_low(), bq.data.px_high()])
    for res in response: 
        print res.name #Prints px_low, px_high 
    
    Print single-item responses by index: 
    print(response[0].name) 

Basics: 
- Import bql: import bql
- Create API connexion: bq = bql.Service()
- Define data items for pricing fields: high = bq.data.px_high() / low = bq.data.px_low()
- Define query for one data item: request = """ get(...) for ([...])""" or request = bql.Request(security, high-low) # if high, low have been defined beforehand 
- Define query for several data items: request = bql.Request('AALPL US Equity', [bq.data.last(), bq.data.high(), bq.data.low()])
- Get query output: response = bq.execute(request)
- Write query as in SQL: 
    
    request = '''
    get(PX_HIGH - PX_LOW)
    for(['AAPL US Equity'])
    '''
    
Contents of query: 
- get() # data you want to retrieve + groupings (e.g. avg profit margin by sector 
- for() # universe containing IDs of several securities, countries, other entities (e.g. in index, select companies whose eps > 0)
- Specify specific date for data item to retrieve: get(px_last(dates = '2018-09-12'))
- Specify range of dates: get(px_last(dates  = (-1M, 0D)))
- Computations within the query: get(px_last/min(px_last(dates = range('-1Y', '0D'))) -1) for (['AAPL US Equity'])
- Retrieve for individual securities: for(['...', 'sec2', 'sec3'])
- Retrieve for members of index: for(members(['index ID']))
- Retrieve all bonds for given issuers: for(bonds(['issuer ID']))
- Retrieve all options tied to securities: for(options(['security ID']))
- Set universe of securities: indu = bq.univ.members('INDU Index')
    
Get datapoint or time series #Depends on param dates. Default to closest date available. To retrieve past datzzaa point, pass single date for start and end 

    bq = bql.Service()
    date_range = bq.func.range('2017-06-05', '2017-06-09') #Dates as string 
    last =  bq.data.px_last(dates = date_range)
    low = bq.data.px_low(dates = date_range)
    request = bql.Request('AAPL US Equity', [last, high, low])
    response = bq.execute(request)
    baql.combined_df(response)

Filter: 
#Filtering only works if you enter, as 2nd expression, a logical test/combined logical tests
# Flow: define your universe, define your filter, apply it, then send your query on the filtered dataset
#Filtering MANDATORY if call universe functions like equitiesUniv() or bondsUniv()
- Filter based on logical condition: for(filter(bonds(['VOD LN Equity']),cpn < 2 and maturity > 2020-12-31))
- Mandatory filter for universe: get(id) for(filter(equitiesUniv(['Active']), market_cap(mode = CACHED) > 100000000 and cntry_of_domicile(mode = CACHED) == 'US'))
- Set criteria as variable: criteria_1 = bq.data.cur_mkt_cap(currency = 'USD') > 50*10**9 / crit1 = bq.data.cpn_typ() == 'FIXED' / crit2 = bq.data.payment_rank() == 'Sr Unsecured'
- Joining a list of criterias: criteria_list = bq.func.and_(criteria1, criteria2)

    Full procedure fo filtering an issuer's bonds: 
    import bql
    bq = bql.Service()
    start_univ = bq.univ.bonds(['IBM US Equity'])
    criteria = bq.data.cpn_typ() == 'FIXED'
    filtered_univ = bq.univ.filter(start_univ, criteria) #Possible to replace one criteria by criteria_list (see above)
    coupon = bq.data.cpn()
    request = bql.Request(filtered_univ, coupon)
    response[0].df() #For a bql object is returned otherwise
    
Groupby: 
# Upon query, if you see entries in different colors, then its different stacks that you need to group before applying calculations on totals 
# Otherwise, subtotals will be displayed
- Query avg over certain days / avg by security type: get(avg(px_last(dates = range(-2D, 0D))))
- Compute avg/median across stacks: get(avg(group(px_last(dates = range(-2D, 0D))))) / get(median(group(prof_margin, by = country_full_name))) / get(median(group(prof_margin, country_full_name)))
- Include avg across stacks in results broken down by security: get(day_to_day_total_return / groupAvg(day_to_day_total_return))
- Groupby GICS sectors: gpby = bq.data.free_cash_flow_yield(fill = 'prev').group(bq.data.gics_sector_name()).avg() #Get avg of fcf yield by gics 
- Apply groupby GICS sectors to universe: request = bql.Request(univ, {'Average FCF Yield per GICS': gpby})
- Get GICS average for each security in the index : gpby = bq.data.free_cash_flow_yield(fill = 'prev').groupavg(grouping_item) # Instead of just giving sector average 
- Create pivot table by grouping by several fields: gpby  = bq.data.cur_mkt_cap(currency = 'EUR').group([grouping_item_1, grouping_item_2]).sum() #Typically: gives total market cap of each sector and country 

Cleaning the extracted dset:     
- Replace NaNs by 0: bq.func.znav(long_term_debt + short_term_debt) #lt debt/st debt = bq.data. [...]

Output of query: 
The query will output the data items you want + associated columns to add context to the datapoints you retrieved 
- Convert to df: response[0].df() if one data item requested / bql.combined_df(response) if several items requested     
    
Plotting the results with bqviz and bqplot
#Really cool since you can refresh, move and save the figure without additional code 
# Check directly the example library for the associated code for each 
# Graphs stylés: 
- KDE Plots pour des histogrames lissés superposés 
- OverUnderPlot pour diagramme en barres +/- x_axis
- Comparison plot: lineplot for two series + OverUnderPlot pour la différence entre les séries (perfect for trackers)
- MultiComparison plot: pour les deux times series, comparison plots sur différents critères (total return, sharpe, ...)
- Interactive line plot: lineplot dynamique (user choisit le critère, start/end etc

Advanced queries notations: 
- #variableName = let() : defines local variables that you can use to simplify other clauses 
- with() : assigns the same parameter value to add data items in yoru query e.g. comparing high/low/open one month ago = use 'with(dates = -1M)'
- preferences() : passes special exec instructions to the query that control how data is returned (e.g. what associated columns are returned in the df output)

## FULL LIST OF DATA ITEMS 
## You won't access to the full list using object. + TAB (too many entries) > Goto "BQL Basics" > "BQL Field, Function and Universe Search" > Run all cells to display search bars 
bq.data.cpn()
bq.data.payment_rank()
bq.data.cpn_typ()
bq.data.cur_mkt_cap(currency = 'USD')
bq.data.px_low(dates = date_range)
bq.data.px_high()
bq.data.px_low(dates = date_range)
bq.data.free_cash_flow_yield(fill='prev')
bq.data.gics_sector_name()
bq.data.bs_lt_borrow()
bq.data.bs_st_borrow()
bq.data.is_eps() #Find income statement eps for a cpy ticker: 

In [ ]:
BQL Fundamental Data
# 3 Key advantages of using BQNT: single field for both reported and estimated data + extract datapoint using revision date, as of date, period end date + data retrieval and analytics 
# Using both fiscal and calendar yields. 
# Point in time data = estimates & actuals (if available) at a certain point in time
# In bold on the notebook,shortcut you may use 
# Never use FPO/FPR together (your reference to a time period cannot be at the same time and )
# AE = fundamental actual v estimate dates (see pdf for further description)
# fa_adjusted = will get you the available fundamental data closest to the date you want 
# Rep object: it's large json file - but formatted as a response 
# You can data.to_excel, data.to_latex, data.to_json after having set up your combined_df 
# fpt = 'A' where A = annual, fpo = '0' for 0 = ;ost recent data available (penultiene periode = 0-1)
# Estimate consensus basics: getting the nb of contributors behind the estimate consensus for instance
# Ex. contributors_revisions: how many revised going up (/ measure of momentum)
# Revision_Window: means that within a year (fpt = A, you look every monthm how many analysts revised their view)
# There is a function for converting bb ticker into fundamental ticker 
# You cqn find an explanation for the translatesymbols. You need this function otherise bb 
# won't retrieve fundamental data otherwise 

Important params
fa_period_type: fpt # Can be Q, BA, BT, S, BS, A, LTM, YTD, BQ, P (fpt = 'A')
fa_period_offset : fpo  #requests both historical values and forward estimates using a single field (e.g fpo = bq.func.range(-1,3) goes one period before and 3 periods after as_of_date value) 
fa_period_reference : fpr #Reference date for the period_end_date (fpr = bq.func.range('2002', '2015')) ou fpr = '2006-09-30'
as of dates: aodd #aod = retrieve data reported as of a specific date  
    
# Forecast revisions, momentumn, vol & backtest your portfolio of picked stocks
import bql 
bq = bql.Service()

def get_screen(ref_date):
    # First you get your funddamental variables and convert to 
    eps = bq.data.is_eps(fpt = 'A', fpo = '1F', dates = bq.func.range('-3m', ref_date), fill = 'prev') # returns a bqlItem  
    eps = bq.func.value(eps, bq.univ.translatesymbols(TARGETIDTYPE = 'FUNDAMENTALTICKER'), mapby = 'lineage')
    eps_rev = bq.func.pct_chg(eps)
    mom = bq.data.total_return(CALC_INTERVAL = bq.func.range('-1y', ref_date)) # not range alone
    vol = bq.data.volatility(CALC_INTERVAL = bq.func.range('-1y', ref_date), PER = 'W')

    # Normalize 
    epsz = bq.func.groupzscore(eps_rev)
    momz = bq.func.groupzscore(mom)
    volz = bq.func.groupzscore(1/vol)
    score = bq.func.avg(epsz, momz, volz) # autonatically discard one variable if doesnt exist
    score_rank = bq.func.grouprank(score)
    # but doesnt really work because its different types of data (epsz is fundamental, rest is stats)

    # Then create your universe 
    index = bq.univ.members('CAC Index', dates = bq.func.range('-1y', ref_date))
    screen = bq.univ.filter(index, score_rank <= 10) # Filters out everything except the top 10

    # Exec the auery - disqble currency check to have func.avg wwork 
    # Why use bq.data.id()? See bql doc 
    req = bql.Request(screen, {'TICKER': bq.data.id()}, preferences = {'currencycheck': 'ignore'}) # gets you chqnge eps estimates per security for that period, for each security 
    res = bq.execute(req)
    top_stocks = bql.combined_df(res)
    return top_stocks

    # How to select the stocks based on the result: we normalize the results and combine the factors
    # We can find the zcore for a group with a built in function!
    # Inspect a zscore: see the notebook 'equity screening' and pdf 
    # How to backtest: straightforwardly enough you just have to change the time for retrieval 
    
    top_stocks = get_screen('2019-05-31').TICKER.tolist()
    # To backtest this selection, we are basically going to build a 1/N portfolio and show total return
    ttr = bq.data.total_return(CALC_INTERVAL = bq.func.range('2019-05-31', '2019-08-30'))
    port_perf = bq.func.avg(bq.func.group(ttr))
    req = bql.Request(top_stocks, {'TTR':port_perf})
    res = bq.execute(req)
    data = bql.combined_df(res)
    data

    # Note that you can loop through requests, or loop through a range to create as many requests
    # you can call the optimizer or export portfolio to port using bql 
    # Better use rolling windows, 5x faster than loops
